<a href="https://colab.research.google.com/github/Praveengovianalytics/genai-prompt_compression/blob/main/prompt_compression_workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install openai -q
 !pip install llmlingua -q
 !pip install accelerate -q
 !pip install llama_index -q
 !pip install optimum auto-gptq -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0

In [ ]:
from openai import OpenAI
client = OpenAI(api_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='In the realm of code, a tale untold,\nLies a secret technique, both new and old,\nWhere functions dance in an endless loop,\nA mystical force, akin to a group.\n\nRecursion, they call it, a curious art,\nA looping wonder, with a fiery heart,\nIt begins with a function, brave and bold,\nIn its quest for answers, unforetold.\n\nImagine a maze, with paths intertwining,\nA mind-boggling puzzle, deeply confining,\nNo end in sight, a labyrinth untamed,\nYet, recursion whispers, "Fear not, unnamed."\n\nThe mighty function, it sets to explore,\nUncharted territory, never seen before,\nBut instead of searching in a straight line,\nIt\'ll take a detour, oh so divine.\n\nOnce tasked with a problem, it does dwell,\nIt splits it in fragments, small and swell,\nAnd with each fragment, a call it makes,\nTo itself, dear friends, a bond it takes.\n\nA self-recursion, a loop within,\nA brave journey to the origin akin,\nAs one call ends, another commences,\nA chain of echo

In [ ]:
from llama_index import (
    VectorStoreIndex,
    download_loader,
    load_index_from_storage,
    StorageContext,
)
WikipediaReader = download_loader("WikipediaReader")
loader = WikipediaReader()
documents = loader.load_data(pages=['Mexican–American_War'])

index = VectorStoreIndex.from_documents(documents,)

retriever = index.as_retriever(similarity_top_k=3)

question = "What were the main outcomes of the war"
contexts = retriever.retrieve(question)

In [ ]:
context_list = [n.get_content() for n in contexts]
context_list

['=== Effect on the United States ===\nIn much of the United States, victory and the acquisition of new land brought a surge of patriotism. Victory seemed to fulfill Democrats\' belief in their country\'s Manifest Destiny. Although the Whigs had opposed the war, they made Zachary Taylor their presidential candidate in the election of 1848, praising his military performance while muting their criticism of the war.\n\nHas the Mexican War terminated yet, and how? Are we beaten? Do you know of any nation about to besiege South Hadley [Massachusetts]? If so, do inform me of it, for I would be glad of a chance to escape, if we are to be stormed. I suppose [our teacher] Miss [Mary] Lyon [founder of Mount Holyoke College] would furnish us all with daggers and order us to fight for our lives ...\nA month before the end of the war, Polk was criticized in a United States House of Representatives amendment to a bill praising Taylor for "a war unnecessarily and unconstitutionally begun by the Presi

In [ ]:
# The response from original prompt
from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate

llm = OpenAI(model="gpt-3.5-turbo-16k")

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)

qa_template = PromptTemplate(template)

# you can create text prompt (for completion API)
prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)

response = llm.complete(prompt)
print(str(response))

The main outcomes of the Mexican-American War were:

1. Acquisition of Territory: The United States gained a significant amount of territory from Mexico, including present-day California, Nevada, Utah, Arizona, New Mexico, and parts of Colorado, Wyoming, Kansas, and Oklahoma. This expansion of territory further fueled the idea of Manifest Destiny and westward expansion.

2. Political Divisions in Mexico: The war exacerbated existing political divisions within Mexico, with different factions vying for power and often fighting against each other instead of focusing on the war effort against the United States.

3. Impact on the United States: The war brought a surge of patriotism in the United States, fulfilling the Democrats' belief in Manifest Destiny. It also led to political divisions within the country, with the Whigs opposing the war but ultimately nominating a military hero, Zachary Taylor, as their presidential candidate.

4. Influence on Military Leaders: Many military leaders wh

In [ ]:
# Setup LLMLingua
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.response_synthesizers import CompactAndRefine
from llama_index.indices.postprocessor import LongLLMLinguaPostprocessor
node_postprocessor = LongLLMLinguaPostprocessor(
    instruction_str="Given the context, please answer the final question",
    target_token=300,
    rank_method="longllmlingua",
    additional_compress_kwargs={
        "condition_compare": True,
        "condition_in_question": "after",
        "context_budget": "+100",
        "reorder_context": "sort",  # enable document reorder,
        "dynamic_context_compression_ratio": 0.3,
    },
)
retrieved_nodes = retriever.retrieve(question)
synthesizer = CompactAndRefine()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
from llama_index.indices.query.schema import QueryBundle

new_retrieved_nodes = node_postprocessor.postprocess_nodes(
    retrieved_nodes, query_bundle=QueryBundle(query_str=question)
)

In [ ]:
original_contexts = "\n\n".join([n.get_content() for n in retrieved_nodes])
compressed_contexts = "\n\n".join([n.get_content() for n in new_retrieved_nodes])
original_tokens = node_postprocessor._llm_lingua.get_token_length(original_contexts)
compressed_tokens = node_postprocessor._llm_lingua.get_token_length(compressed_contexts)
print(compressed_contexts)
print()
print("Original Tokens:", original_tokens)
print("Compressed Tokens:", compressed_tokens)
print("Compressed Ratio:", f"{original_tokens/(compressed_tokens + 1e-5):.2f}x")

Has the Mexican, how Are beaten you know of any bes Hadley [assachusetts], me of, I would be glad of escape,ed [our teacher Missary Lyoner ofoke College]ish us allgers and order us fight for our lives ... month before the of the, Pol was critic a United States Housement billising for "a wararily and un begun by President of the.", whichman Abraham played an important role with his Sp Resolutions, followedal of thesings, including to Presidentk. party with alligs supportingment. Lincolns won lukearm support fellow Whigs Illinois counter attacked by who r pro-war Illinoiss Spotutions state Illinois by his his.While Ralphdoerson rejected waras means of achieving toward of the war he: "The United will conquer, man swows, which brings him down. Mexico will poison us." He later accepted that the are about bycred means War historian James M an of Pul winning War history to theAmerican, "Mison Us Mcersonues that the Mexican and was a key territorial event leadup to the Civil WarVeterans of the

# Reset the session

# Using llmlingua Library - Oneline meeting dataset

In [ ]:
# Install dependency.
!pip install datasets -q

In [ ]:
# Download the original prompt and dataset
from datasets import load_dataset
dataset = load_dataset("lytang/MeetingBank-transcript")["train"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
# select an example from MeetingBank
contexts = dataset[1]["source"]

In [ ]:
print(contexts)

Speaker 4: We're going to hear all of the presentations at once. And then after we go through all the presentations, we'll do all the all of the questions at once and then any any public comment, and we'll go from there. Thank you.
Speaker 7: All right. Thank you, Mr. Mayor. It was our pleasure to give you an overview of the budget last Tuesday, we transmitted that officially to the mayor and city council and the public. And now we're looking to really dove deeper in some of the key departments. Today, we want to go through three key departments, the fire department. Then we'll go through our police department and then Parks, Recreation and Marine. So I will turn it over to Chief Espino and his team to go through the fire department presentation first followed by police, followed by Parks and Rec.
Speaker 1: It's.
Speaker 6: Thank you, Mr. Monaco. Good evening. Mayor Garcetti and members of the City Council. This evening, I will be providing a brief overview of the fire department, hig

In [ ]:
question = "Question: How much did the crime rate increase last year?\nAnswer:"
reference = "5.4%"

# Prepare the prompt to ChatGPT 4 Preview

In [ ]:
from openai import OpenAI
client = OpenAI(api_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

In [ ]:
# The response from original prompt, using GPT-4-32k
import json
prompt = "\n\n".join([contexts, question])

message = [
    {"role": "user", "content": prompt},
]

request_data = {
    "messages": message,
    "max_tokens": 100,
    "temperature": 0,
    "top_p": 1,
    "n": 1,
    "stream": False,
}

In [ ]:
completion = client.chat.completions.create(
  model="gpt-4-1106-preview",
 **request_data
)
#print(completion.choices[0].message)

In [ ]:
completion

ChatCompletion(id='chatcmpl-8f0C0EwGq39lXJvdtwxNKD2mOnsgF', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='According to the information provided in the dialogue, the violent crime rate experienced a 5.4% increase year to date. However, it was also mentioned that at the beginning of the year, there was a 17.4% increase in violent crime, which has since decreased to the current 5.4% increase.', role='assistant', function_call=None, tool_calls=None))], created=1704782880, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_168383a679', usage=CompletionUsage(completion_tokens=66, prompt_tokens=30096, total_tokens=30162))

# LLMLingua in Action

In [ ]:
# Setup LLMLingua
from llmlingua import PromptCompressor
llm_lingua = PromptCompressor("TheBloke/Llama-2-7b-Chat-GPTQ", model_config={"revision": "main"})
#llm_lingua = PromptCompressor()

## Or use the quantation model, like TheBloke/Llama-2-7b-Chat-GPTQ, only need <8GB GPU memory.
## Before that, you need to pip install optimum auto-gptq


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# 200 Compression
compressed_prompt = llm_lingua.compress_prompt(
    contexts.split("\n"),
    instruction="",
    question=question,
    target_token=200,
    condition_compare=True,
    condition_in_question='after',
    rank_method='longllmlingua',
    use_sentence_level_filter=False,
    context_budget="+100",
    dynamic_context_compression_ratio=0.4, # enable dynamic_context_compression_ratio
    reorder_context="sort"
)

In [ ]:
message = [
    {"role": "user", "content": compressed_prompt["compressed_prompt"]},
]

request_data = {
    "messages": message,
    "max_tokens": 100,
    "temperature": 0,
    "top_p": 1,
    "n": 1,
    "stream": False,
}

completion = client.chat.completions.create(
  model="gpt-4-1106-preview",
 **request_data
)

print(completion)

ChatCompletion(id='chatcmpl-8f0MTO2m610I4otg8TK2g7q4F42aW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The provided text does not contain clear information about the overall crime rate increase from the previous year. It mentions a figure of "17.4%" but does not specify what this percentage refers to or if it is related to an increase or a current statistic. Additionally, the text is fragmented and lacks context, making it impossible to accurately determine the crime rate increase from the given information. If you have more context or a specific report to refer to, please provide that for an accurate answer.', role='assistant', function_call=None, tool_calls=None))], created=1704783529, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_168383a679', usage=CompletionUsage(completion_tokens=97, prompt_tokens=155, total_tokens=252))


In [ ]:
completion

ChatCompletion(id='chatcmpl-8f0MTO2m610I4otg8TK2g7q4F42aW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The provided text does not contain clear information about the overall crime rate increase from the previous year. It mentions a figure of "17.4%" but does not specify what this percentage refers to or if it is related to an increase or a current statistic. Additionally, the text is fragmented and lacks context, making it impossible to accurately determine the crime rate increase from the given information. If you have more context or a specific report to refer to, please provide that for an accurate answer.', role='assistant', function_call=None, tool_calls=None))], created=1704783529, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_168383a679', usage=CompletionUsage(completion_tokens=97, prompt_tokens=155, total_tokens=252))